## 2. Preparação para Tokenização (Pré-processamento)

* **Input**: CSV contendo todos os enunciados, alternativas e gabaritos

* **Output**: CSV com as colunas pré-processadas -> *numero_questao*, *enunciado*, *alternativas*,*nu_param_B*, *gabarito*, *ano*, *enunciado_limpo* e *alternativas_limpo*

In [73]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords

In [74]:
# Download stopwords (palavras de parada)
nltk.download('stopwords')
stop_words = set(stopwords.words('portuguese'))

def clean_text(text):
    """Processando texto de acordo com o Protocolo Primi (2021), citado no artigo"""
    if not isinstance(text, str):  # Garantir que texto é string
        return ""
    
    words = re.findall(r'\b[a-zA-Zà-úÀ-ÚüÜ]+\b', text.lower())  # 1. Dividir item em palavras
    words = [word for word in words if word not in stop_words]  # 2. Remover stopwords
    return " ".join(sorted(set(words)))  # 3. Remover palavras duplicadas

def clean_alternatives(alt_text):
    """Processa as alternativas mantendo as letras (A:, B:, etc.) e limpando o conteúdo."""
    if not isinstance(alt_text, str):
        return ""
    
    alternatives = re.split(r'(?=[A-E]: )', alt_text)  # Divide mantendo os identificadores
    cleaned_alts = []
    
    for alt in alternatives:
        if ": " in alt:
            key, value = alt.split(": ", 1)  # Separa a letra da alternativa do conteúdo
            cleaned_value = clean_text(value)  # Aplica limpeza somente no conteúdo
            cleaned_alts.append(f"{key}: {cleaned_value}")  # Mantém o formato original
    
    return "; ".join(cleaned_alts)  # Junta novamente as alternativas


PADROES_DESCRICAO = [
    r"Descrição da Figura",
    r"Descrição da imagem",
    r"Descrição do mapa",
]

regex_descricao = r"(" + "|".join(PADROES_DESCRICAO) + r".*)"

def separar_descricao(row):
    alternativas = row['alternativas']
    
    match = re.search(regex_descricao, alternativas, re.IGNORECASE)
    
    if match:
        descricao = match.group(1).strip()
        alt_limpa = alternativas[:match.start()].strip()
        nova_questao = row['enunciado'].strip() + " " + descricao
        return pd.Series([alt_limpa, nova_questao])
    else:
        return pd.Series([alternativas, row['enunciado']])




def separate_distractors(row):
    items: list = [
        alternativa.strip()
        for alternativa in row["alternativas"].split(";")
        if alternativa.strip()
    ]

    mapping_items = {}
    for item in items:
        letra, texto = item.split(":", 1)
        mapping_items[letra.strip()] = texto.strip()

    # Extraindo texto do gabarito
    gabarito: str = row["gabarito"]
    row["gabarito_texto"] = mapping_items.get(gabarito.strip(), "")

    # Coletando alternativas distratoras
    row["distratores"] = "; ".join(
        [texto for letra, texto in mapping_items.items() if letra != gabarito]
    )

    return row

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/gustavo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [75]:
# separa os casos onde as descrições estão nos enunciados

df[['alternativas', 'enunciado']] = df.apply(separar_descricao, axis=1)

In [76]:
# separa os distratores

df = df.apply(separate_distractors, axis=1)

df.head()

,numero_questao,gabarito,NU_PARAM_A,nu_param_B,NU_PARAM_C,ANO,enunciado,alternativas,gabarito_texto,distratores,enunciado_limpo,alternativas_limpo,gabarito_texto_limpo,distratores_limpo
0,1,C,3.43894,0.97831,0.10855,2017,"No império africano do Mali, no século XIV, Tombuctu foi centro de um comércio internacional onde tudo era negociado — sal, escravos, marﬁm etc. Havia também um grande comércio de livros de história, medicina, astronomia e matemática, além de grande concentração de estudantes. A importância cultural de Tombuctu pode ser percebida por meio de um velho provérbio: “O sal vem do norte, o ouro vem do sul, mas as palavras de Deus e os tesouros da sabedoria vêm de Tombuctu”.Uma explicação para o dinamismo dessa cidade e sua importância histórica no período mencionado era o(a)",A: isolamento geográﬁco do Saara ocidental; B: exploração intensiva de recursos naturais; C: posição relativa nas redes de circulação; D: tráﬁco transatlântico de mão de obra servil; E: competição econômica dos reinos da região,posição relativa nas redes de circulação,isolamento geográﬁco do Saara ocidental; exploração intensiva de recursos naturais; tráﬁco transatlântico de mão de obra servil; competição econômica dos reinos da região,africano além astronomia centro cidade comércio concentração cultural dessa deus dinamismo escravos estudantes etc explicação grande havia história histórica importância império internacional livros mali matemática medicina meio mencionado negociado norte onde ouro palavras percebida período pode provérbio sabedoria sal sul século tesouros tombuctu tudo velho vem vêm xiv,A: isolamento ocidental saara; B: exploração intensiva naturais recursos; C: circulação posição redes relativa; D: mão obra servil transatlântico; E: competição econômica região reinos,circulação posição redes relativa,competição econômica exploração intensiva isolamento mão naturais obra ocidental recursos região reinos saara servil transatlântico
1,2,D,3.00837,0.49169,0.13877,2017,"Após a Declaração Universal dos Direitos Humanos pela ONU, em 1948, a Unesco publicou estudos de cientistas de todo o mundo que desqualiﬁcaram as doutrinas racistas e demonstraram a unidade do gênero humano. Desde então, a maioria dos próprios cientistas europeus passou a reconhecer o caráter discriminatório da pretensa superioridade racial do homem branco e a condenar as aberrações cometidas em seu nome.A posição assumida pela Unesco, a partir de 1948, foi motivada por acontecimentos então recentes, dentre os quais se destacava o(a)",A: ataque feito pelos japoneses à base militar americana de Pearl Harbor.; B: desencadeamento da Guerra Fria e de novas rivalidades entre nações; C: morte de milhões de soldados nos combates da Segunda Guerra Mundial.; D: execução de judeus e eslavos presos em guetos e campos de concentração nazistas; E: lançamento de bombas atômicas em Hiroshima e Nagasaki pelas forças norte-americanas,execução de judeus e eslavos presos em guetos e campos de concentração nazistas,ataque feito pelos japoneses à base militar americana de Pearl Harbor.; desencadeamento da Guerra Fria e de novas rivalidades entre nações; morte de milhões de soldados nos combates da Segunda Guerra Mundial.; lançamento de bombas atômicas em Hiroshima e Nagasaki pelas forças norte-americanas,aberrações acontecimentos após assumida branco caráter cientistas cometidas condenar declaração demonstraram dentre desde destacava direitos discriminatório doutrinas então estudos europeus gênero homem humano humanos maioria motivada mundo nome onu partir passou posição pretensa próprios publicou quais racial racistas recentes reconhecer superioridade todo unesco unidade universal,A: americana ataque base feito harbor japoneses militar pearl; B: desencadeamento fria guerra nações novas rivalidades; C: combates guerra milhões morte mundial segunda soldados; D: campos concentração eslavos execução guetos judeus nazistas presos; E: americanas atômicas bombas forças hiroshima lançamento na

In [77]:
# Aplicando pré-processamento no enunciado e alternativas
df["enunciado_limpo"] = df["enunciado"].apply(clean_text)
df["alternativas_limpo"] = df["alternativas"].apply(clean_alternatives)
df["gabarito_texto_limpo"] = df["gabarito_texto"].apply(clean_text)
df["distratores_limpo"] = df["distratores"].apply(clean_text)

# Save cleaned CSV
df.to_csv("../data/cleaned_data.csv", index=False)
print("Processing complete. File saved as 'cleaned_data.csv'.")


Processing complete. File saved as 'cleaned_data.csv'.
